# Basic Example
Make a basic likelihood (and posterior) example for *What is a Measurement?*

## Author:
- **David W. Hogg** (NYU) (MPIA) (Flatiron)

## Notes:
- This is part of the *What is a Measurement?* project. Copyright the author.

## Bugs:
- Global variables `N` and `prior_bounds`.
- The number of parameters (4) hard-coded all over the place.

In [ ]:
import numpy as np
import pylab as plt
import matplotlib as mpl
import scipy.optimize as op
import emcee

In [ ]:
mpl.rcParams['figure.figsize'] = [3.0, 3.0]

In [ ]:
N = 16
prior_bounds = np.array([[1., 3.], [1., 2.], [0., 2. * np.pi], [2., 4.]])

def expectation(ts, pars):
    om, amp, phi, y0 = pars
    return y0 + amp * np.cos(om * ts + phi)

def make_fake_data(seed=17):
    rng = np.random.default_rng(seed)
    ts = np.sort(7. * rng.uniform(size=N))
    ivars = 0.5 + 0.5 * rng.uniform(size=N)
    truepars = np.zeros(4) + np.nan
    for i in range(4):
        truepars[i] = rng.uniform(low=prior_bounds[i,0],
                                  high=prior_bounds[i,1])
    return ts, expectation(ts, truepars) + rng.normal(size=N) / np.sqrt(ivars), ivars, truepars

In [ ]:
ts, ys, ivars, true_pars = make_fake_data()
print(ts.shape, ys.shape, true_pars)

In [ ]:
def plot(ts, ys, ivars, true_pars, ml_pars, samples, title):
    plt.errorbar(ts, ys, yerr=1./np.sqrt(ivars), fmt="ko")
    plot_ts = np.linspace(0., 7., 1000)
    if samples is not None:
        for sample in samples:
            plt.plot(plot_ts, expectation(plot_ts, sample), "r-", lw=1, alpha=0.45)
    if true_pars is not None:
        plt.plot(plot_ts, expectation(plot_ts, true_pars), "b-", lw=1, alpha=0.45)
    if ml_pars is not None:
        plt.plot(plot_ts, expectation(plot_ts, ml_pars), "r-", lw=2, alpha=0.9)
    plt.xlabel("time")
    plt.ylabel("data value")
    plt.title(title)

plot(ts, ys, ivars, true_pars, None, None, "data and true expectation")

In [ ]:
def negative_log_likelihood(pars, ts, ys, ivars):
    return 0.5 * np.sum(ivars * (ys - expectation(ts, pars)) ** 2)

In [ ]:
res = op.minimize(negative_log_likelihood, true_pars, args=(ts, ys, ivars))
print(res)
ml_pars = np.zeros(4) + np.nan
ml_pars_covar = np.zeros((4,4)) + np.nan
if res.success:
    ml_pars = res.x
    ml_pars_covar = res.hess_inv
print(ml_pars)

In [ ]:
plot(ts, ys, ivars, true_pars, ml_pars, None,
     "maximum likelihood")

In [ ]:
def negative_partial_log_likelihood(al, om, ts, ys, ivars):
    pars = np.append(om, al)
    return 0.5 * np.sum(ivars * (ys - expectation(ts, pars)) ** 2)

def negative_profile_log_likelihood(om, ts, ys, ivars, guess):
    res = op.minimize(negative_partial_log_likelihood, guess, args=(om, ts, ys, ivars))
    if res.success:
        return res.fun
    return np.nan

In [ ]:
oms = np.linspace(0.5, 3.5, 100)
plls = np.array([negative_profile_log_likelihood(om, ts, ys, ivars, true_pars[1:]) for om in oms])
print(plls.shape)

In [ ]:
plt.plot(oms, -1. * plls, "k-")
plt.axvline(true_pars[0], c="b", lw=1, alpha=0.45)
plt.axvline(ml_pars[0], c="r", lw=2, alpha=0.9)
ylim = plt.ylim()
plt.fill_betweenx(ylim, [ml_pars[0] - np.sqrt(ml_pars_covar[0,0])] * 2,
                        [ml_pars[0] + np.sqrt(ml_pars_covar[0,0])] * 2,
                 color="r", alpha=0.20)
plt.ylim(ylim)
plt.xlabel("angular frequency")
plt.ylabel("log likelihood (plus offset)")
plt.title("profile log likelihood")

In [ ]:
ntrial = 64
many_true_pars = np.zeros((ntrial, 4)) + np.nan
many_ml_pars = np.zeros((ntrial, 4)) + np.nan
many_ml_pars_covar = np.zeros((ntrial, 4, 4)) + np.nan
for trial in range(ntrial):
    ts, ys, ivars, tp = make_fake_data(seed=trial)
    many_true_pars[trial] = tp
    res = op.minimize(negative_log_likelihood, tp, args=(ts, ys, ivars))
    if res.success:
        many_ml_pars[trial] = res.x
        many_ml_pars_covar[trial] = res.hess_inv
print(np.sum(np.isnan(many_ml_pars[:, 0])))

In [ ]:
plt.plot([-10, 10], [-10, 10], "k-", alpha=0.45)
plt.errorbar(many_true_pars[:,0], many_ml_pars[:,0],
             yerr=np.sqrt(many_ml_pars_covar[:,0,0]), fmt="k.",
             alpha=0.9)
plt.axis("equal")
plt.xlim(0.5, 3.5)
plt.ylim(0.5, 3.5)
plt.xlabel("true angular frequency")
plt.ylabel("maximum-likelihood frequency")
plt.title("maximum likelihood is unbiased")

In [ ]:
def log_prior(pars):
    for i in range(4):
        if pars[i] < prior_bounds[i, 0]:
            return -np.Inf
        if pars[i] > prior_bounds[i, 1]:
            return -np.Inf
    return 0.

def log_posterior(pars, ts, ys, ivars):
    lnpi = log_prior(pars)
    if np.isfinite(lnpi):
        return lnpi - negative_log_likelihood(pars, ts, ys, ivars)
    return lnpi

In [ ]:
def sample_one_trial(seed, return_everything=False):
    print("sample_one_trial():", seed)
    ts, ys, ivars, tp = make_fake_data(seed=seed)
    ndim, nwalkers = 4, 20
    p0 = tp + 0.01 * np.random.randn(nwalkers, ndim)
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=[ts, ys, ivars])
    # burn in
    sampler.run_mcmc(p0, 6000)
    p0 = (sampler.get_chain())[-1]
    # print("sample_one_trial(): number of bad values", np.sum(np.logical_not(np.isfinite(p0))))
    # do final run
    sampler.run_mcmc(p0, 50)
    ps = sampler.get_chain(flat=True)
    # get statistics
    posterior_mean = np.mean(ps, axis=0)
    dp = ps - posterior_mean[None, :]
    posterior_covar = np.mean(dp[:, :, None] * dp[:, None, :], axis=0)
    if return_everything:
        return tp, posterior_mean, posterior_covar, ts, ys, ivars, ps
    return tp, posterior_mean, posterior_covar

In [ ]:
true_pars2, post_pars, post_pars_covar, ts, ys, ivars, post_samples = sample_one_trial(17, return_everything=True)
print(post_pars.shape, post_pars_covar.shape, post_samples.shape)

In [ ]:
plot(ts, ys, ivars, true_pars2, None, post_samples[:12],
     "posterior samples")

In [ ]:
many_true_pars2 = np.zeros((ntrial, 4)) + np.nan
many_post_pars = np.zeros((ntrial, 4)) + np.nan
many_post_pars_covar = np.zeros((ntrial, 4, 4)) + np.nan
for trial in range(ntrial):
    many_true_pars2[trial], many_post_pars[trial], many_post_pars_covar[trial] = sample_one_trial(trial)

In [ ]:
plt.plot([-10, 10], [-10, 10], "k-", alpha=0.45)
plt.errorbar(many_true_pars2[:,0], many_post_pars[:,0],
             yerr=np.sqrt(many_post_pars_covar[:,0,0]), fmt="k.",
             alpha=0.9)
plt.axis("equal")
plt.xlim(0.5, 3.5)
plt.ylim(0.5, 3.5)
plt.xlabel("true angular frequency")
plt.ylabel("posterior mean frequency")
plt.title("posterior inference is biased")